<a href="https://colab.research.google.com/github/askoki/football-mock-data/blob/main/%5Bworksheet%5D_DSC_Europe_football_workshop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Think You Can Handle Football Player Data? Join the Club’s Data Lab

Welcome to the Data Lab session, where we’ll walk through the exciting world of football player data analysis. Get ready to:

1. **Load and Clean GPS Datasets**  
   - Learn how to import and prepare GPS data for analysis.
   - Fix formatting issues and create new features.

2. **Visualize Player Data**  
   - Learn how to plot key metrics and trends using simple and powerful visualizations.

3. **Calculate and Visualize Exponential Weighted Moving Averages (EWMA)**  
   - Understand how to smooth performance data over time and visualize it through the season.

4. **Analyze Winter Conditioning Period**  
   - Dive into the winter conditioning phase to visualize the team’s workload and track progress.

5. **Create a Player Progress Report**  
   - Build a report that allows coaches to compare a player's current performance with their season-best data.

By the end of this session, you’ll be able to analyze and visualize player data like a pro, helping coaches and analysts make smarter decisions for the season.
---

# 1. Dataset Loading and Initial Exploration

---

## 1.1 Data Loading

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
DATA_URL = 'https://raw.githubusercontent.com/askoki/football-mock-data/refs/heads/main/FC_Europe_dataset.csv'
df = pd.read_csv(DATA_URL)

In [ ]:
df.head()

In [ ]:
df.describe()

## 1.2 Dataset Inspection Task

In this session, we’ll conduct an initial exploratory analysis of the dataset, focusing on key structural insights:

1. **Player Count**  
   - Determine the total number of unique players represented in the dataset.

2. **Data Points per Player**  
   - Identify the number of data entries associated with each player, providing a breakdown of data density across players.

3. **Session Coverage**  
   - Calculate the number of distinct days where sessions took place, giving insight into session frequency and consistency over time.

In [ ]:
df.player.unique()

In [ ]:
df.player.unique().shape

In [ ]:
df.player.value_counts()

## 1.3 Date Inspection and Formatting Correction

Next, we'll examine the date data for any inconsistencies or errors in formatting.

📌 **Task:** Review the date entries to detect any anomalies, such as incorrect formats, missing dates, or out-of-sequence entries.

### Hint:
To help identify any issues with the date data, try using some of the functions we've previously applied to inspect the player data:
- Use `describe()` to get an overview of the date field.
- Apply `min()` and `max()` to find the earliest and latest dates, helping to spot any unexpected outliers.
- Use `unique()` with `shape` to check the variety of date formats. Do they make sense?

By addressing these potential formatting issues, we’ll improve the dataset's reliability and prepare it for accurate analysis in further stages.

# 2. Data Cleaning and Date Formatting

---

## 2.1 Fixing Date Format

We’ll standardize the dates to a uniform format (e.g., `YYYY-MM-DD`) to ensure consistency across the dataset.

In [ ]:
df.date.dtypes

In [ ]:
df.date.iloc[0]

In [ ]:
df.date = df.date.str.replace('.', '-')

In [ ]:
df.date.unique().shape

In [ ]:
df.head(3)

## 2.2 Date Sorting

In [ ]:
df = df.sort_values('date')
df.head()

In [ ]:
df.reset_index(inplace=True, drop=True)
df.head()

## 2.3 Data cleaning

### Goal: Inspecting Session Types

Our first step in data cleaning is to understand the distribution of session types in the dataset. We’ll start by exploring which session types are present and how frequently each occurs.

**📌 Task**: Use the following command to check the frequency of each session type:
```python
df.session_type.value_counts()
```

### 2.3.1 Adjusting Session Types Based on Coach Feedback

Based on the coach’s review, certain `OFFICIAL MATCHES` were actually friendly matches during specified periods. We need to update these entries to ensure accurate categorization.

#### Coach’s Instruction:
- Update all sessions labeled as `OFFICIAL MATCHES` in the date ranges:
  - **15.6.2024 to 26.7.2024**
  - **1.1.2025 to 29.1.2025**

These sessions should be reclassified as `FRIENDLY MATCHES`.

**📌 Task:**  
The `replace_session_type` function is already implemented to perform this update. Your task is to try running the function and identify any issues that may arise.

In [ ]:
from datetime import datetime, date

def replace_session_type(df: pd.DataFrame, start_date: datetime.date, end_date: datetime.date) -> pd.DataFrame:
    df.loc[
        (df['date'] >= start_date) &
        (df['date'] <= end_date) &
        (df['session_type'] == 'OFFICIAL MATCH'),
        'session_type'
    ] = 'FRIENDLY MATCH'

    return df


In [ ]:
df = replace_session_type(df, date(2024, 6, 15), date(2024, 7, 26))

### 2.3.2 Inspecting Data Types and Fixing Date Format

In [ ]:
df.date.iloc[0]

In [ ]:
type(df.date.iloc[0])

In [ ]:
df.date = pd.to_datetime(df.date).dt.date

In [ ]:
df.date.iloc[0]

In [ ]:
type(df.date.iloc[0])

In [ ]:
df = replace_session_type(df, date(2024, 6, 15), date(2024, 7, 26))

**📌 Task:**
- Update Training Labels for the Winter Preseason Period – **1.1.2025 to 29.1.2025**
- Check which session types are now present in the dataset?

# 3. Average Load Calculation

---
At the end of the season, coaches rely on our analysis to evaluate the training load throughout the season. This helps identify areas where fitness levels can be improved and where injury risk can be reduced.



In [ ]:
df.groupby('date').size()

In [ ]:
df.columns

## 3.1  What was the average load across the season?

We need the following in order to achieve that:

1. Compute the daily average load for across all sessions (e.g., training or match).
2. Visualize the results using a plot.

In [ ]:
df.groupby('date').agg({
    'session_type': 'first',
    'duration': 'mean',
    'total_distance_m': 'mean'
})

**📌 Task:** Can you spot the issue?

### 3.1.1 Side Quest: Fix Non-Numeric Data Issues

In [ ]:
df.duration.iloc[0]

In [ ]:
df.duration.dtypes

In [ ]:
df.loc[:, 'duration_min'] = pd.to_timedelta(df.duration).dt.total_seconds() / 60
df.loc[:, 'duration_min'] = df.duration_min.astype(int)

In [ ]:
df = df.drop(columns='duration')

In [ ]:
df.head(3)

### 3.1.2 Back to Computing Daily Average

📌 **Task:** Use the previously demonstrated `groupby` method to calculate the average per date and store the results in `avg_df`. The aggregation should be performed as follows:

- `session_type`: first
- `duration_min`: mean
- `total_distance_m`: mean



## 3.2 Visualization Time

Great job on the data cleaning! Now, let's showcase our results using the prepared `plot_gps_param` function.

In [ ]:
import matplotlib.dates as mdates
plt.style.use('ggplot')

def plot_gps_param(df: pd.DataFrame, param_name: str, title: str) -> None:
  fig, ax = plt.subplots(figsize=(14, 3), sharex=True)
  fig.suptitle(title, fontsize=12, color='Black')

  x = avg_df.date.values
  y = avg_df[param_name].values

  ax.bar(x, y, label=param_name)

  ax.xaxis.set_major_locator(mdates.WeekdayLocator(byweekday=mdates.MO))
  ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
  plt.xticks(rotation=45)
  plt.legend()
  plt.show()

In [ ]:
plot_gps_param(avg_df, 'total_distance_m', 'Average across the season')

### 3.2.1 Visualizing the Difference Between Trainings and Matches

The coach has requested that we visualize the difference between training sessions and matches. To achieve this, we need to:

1. Add a boolean column `is_match` to easily differentiate between the two.
2. Modify the `plot_gps_param` function to visualize the data using two distinct bar plots for training sessions and matches.

In [ ]:
avg_df.session_type.value_counts()

In [ ]:
avg_df.loc[:, 'is_match'] = avg_df.apply(lambda r: False if r.session_type == 'FULL TRAINING' else True, axis=1)
avg_df.is_match.value_counts()

In [ ]:
ggplot_colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

primary_color = ggplot_colors[0]
secondary_color = ggplot_colors[1]

In [ ]:
def plot_gps_param(df: pd.DataFrame, param_name: str, title: str) -> None:
  fig, ax = plt.subplots(figsize=(14, 4), sharex=True)
  fig.suptitle(title, fontsize=16)

  training_df = df[df.is_match == False]
  match_df = df[df.is_match == True]

  ax.bar(training_df.date.values, training_df[param_name].values, color=primary_color, label='Trainings')
  ax.bar(match_df.date.values, match_df[param_name].values, color=secondary_color, label='Matches')

  ax.xaxis.set_major_locator(mdates.WeekdayLocator(byweekday=mdates.MO))
  ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
  plt.xticks(rotation=45)
  plt.legend()
  plt.show()

In [ ]:
plot_gps_param(avg_df, 'total_distance_m', 'Average across the season')

## 3.3 Exponential Weighted Moving Average (EWMA)

Training sessions vary, and the load fluctuates as the season progresses. To get a clearer overview, we need to smooth the data. Since more recent training load values are more relevant than older ones (e.g., the load from three months ago), the Exponential Weighted Moving Average (EWMA) is a commonly used technique. In this section, we'll explore how EWMA works and how it can help us better understand the data.

### 3.3.1 EWMA Example

In [ ]:
DATA_LENGTH = 30

data_x = np.arange(DATA_LENGTH)
data_y = np.random.rand(1, data_x.shape[0]) * 100
data_y = data_y.astype(int)[0]

example_df = pd.DataFrame({'x': data_x, 'y': data_y})

In [ ]:
plt.style.use('default')

In [ ]:
plt.figure(figsize=(4,3))
plt.bar(example_df.x, example_df.y)
plt.show()

In [ ]:
ewma_window_5 = example_df.y.ewm(5).mean().values
ewma_window_5

In [ ]:
plt.figure(figsize=(4,3))
plt.bar(example_df.x, example_df.y, label='Mock Data')
plt.plot(example_df.x, ewma_window_5, color='black', linestyle='--', label='EWMA (window=5)')
plt.legend()
plt.show()

📌 **Task:** Modify the `plot_gps_param` function to plot the Exponential Weighted Moving Average (EWMA) values for the specified parameter (`param_name`) across the season, using a 14-day window. You can plot the values with the following code, just replace `x` and `y` with the corresponding values:

```python
ax.plot(x, y, color='black', linestyle='--', label='EWMA (14 days)')


In [ ]:
def plot_gps_param(df: pd.DataFrame, param_name: str, title: str) -> None:
  fig, ax = plt.subplots(figsize=(14, 4), sharex=True)
  fig.suptitle(title, fontsize=16)

  training_df = df[df.is_match == False]
  match_df = df[df.is_match == True]

  ax.bar(training_df.date.values, training_df[param_name].values, color=primary_color, label='Trainings')
  ax.bar(match_df.date.values, match_df[param_name].values, color=secondary_color, label='Matches')
  # ---------------- ADD CODE HERE ----------------
  x = df.date.values
  y = # you need to write it
  # plotting
  # ----------------  END OF CODE ----------------
  ax.xaxis.set_major_locator(mdates.WeekdayLocator(byweekday=mdates.MO))
  ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
  plt.xticks(rotation=45)
  plt.legend()
  plt.show()

In [ ]:
plot_gps_param(avg_df, 'total_distance_m', 'Average across the season')

# 4. Weekly Data Aggregation for Team

---

Now that we have an overall picture of the season, let's shift focus to a more frequent scenario. Imagine being in the middle of the season, where you need to estimate the load for the upcoming week. We will place ourselves at the end of the winter preparation period (from 1st January 2025 to 29th January 2025) and explore how to visualize cumulative training load to assist strength and conditioning coaches in planning their work.

In [ ]:
df.loc[:, 'is_match'] = df.apply(lambda r: False if r.session_type == 'FULL TRAINING' else True, axis=1)
zoom_df = df[df.date < date(2025, 1, 29)]
zoom_df.tail(5)

## 4.1 Selecting the Time Span

In [ ]:
from datetime import timedelta

date_until = zoom_df.date.max()
date_from = date_until- timedelta(days=28)
weeks4_df = zoom_df[(zoom_df.date >= date_from) & (zoom_df.date <= date_until)]
weeks4_df

## 4.2 Average Load per Week for the Last 4 Weeks

📌 **Task:** Begin by grouping the sessions by date and use only the `cols2inspect` columns for aggregation. You can do this using the following code:
```python
weeks4_df.groupby('date', as_index=False)[cols2inspect]
```
Then, aggregate all parameters by taking the mean using `.agg('mean')` and store it into variable `day_mean_df`.

---



In [ ]:
cols2inspect = [
    'total_distance_m', 'HSR_m', 'sprint_m', 'acc_cnt', 'dec_cnt', 'player_load'
]

### 4.2.1 Adding Week Numbers for Grouping

Next, we will add a week number column to the DataFrame, allowing us to group the data by week.

In [ ]:
day_mean_df.loc[:, 'week'] = day_mean_df.date.apply(lambda r: r.isocalendar()[1])

In [ ]:
day_mean_df

Finally, we can group the data by week and calculate the sum of the values.

In [ ]:
weeks4_df = day_mean_df.drop(columns='date').groupby('week', as_index=False).agg(np.sum)
weeks4_df

In [ ]:
plt.bar(weeks4_df.week, weeks4_df.total_distance_m)
plt.xlabel('Weeks')
plt.ylabel('Total Distance (m)')

## 4.3 More sophisticated approach

Let's show the numbers in comparison to top 5 matches that the team has played up until this point in season. This will allow coach for easier understanding of these numbers in comparison to the desired output and that is match perofrmance.

In [ ]:
avg_df = df.groupby('date')[cols2inspect + ['is_match']].agg(np.mean).reset_index()
avg_df.head(3)

### 4.3.1 Calculating the Mean of the Top 5 Matches

Now, let's calculate the mean of the top 5 matches the team has played to date.

In [ ]:
match_ref_df = pd.DataFrame(index=[0])
match_df = avg_df[avg_df.is_match == True]
for col in cols2inspect:
  best5_df = match_df.sort_values(col, ascending=False)[col].values[:5]
  match_ref_df.loc[:, col] = best5_df.mean()
match_ref_df

### 4.3.2 Estimating Weekly Load Distribution Using `match_ref_df`

For each observed parameter, we will divide the values by those in the `match_ref_df`. The results will be displayed in a stacked bar chart with four levels stacked on top of each other. To achieve this, we will divide the resulting values by 4.



In [ ]:
relative_df = weeks4_df.copy()
for param in cols2inspect:
    relative_df.loc[:, param] /= match_ref_df.loc[:, param].values
    # We will be stacking four parameters on the graph later so we need to adjust the values.
    NUMBER_OF_ITEMS_STACKED = 4
    relative_df.loc[:, param] /= NUMBER_OF_ITEMS_STACKED
relative_df = relative_df.round(2)

In [ ]:
relative_df

### 4.3.3 Plotting time

In [ ]:
DARK_GRAY = '#7f7f7f'
LIGHT_GRAY = '#d9d9d9'
fill_design = [
    (DARK_GRAY, True, None, 0.6),
    (LIGHT_GRAY, True, None, 0.6),
    (None, False, '///', 0.6),
    (None, False, '..', 0.2),
    (None, False, 'OO', 0.2),
    (None, False, '', 0.2)
]

In [ ]:
plt.style.use('default')

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from datetime import timedelta
from matplotlib.ticker import MaxNLocator


fig, ax = plt.subplots(figsize=(10, 6), facecolor='white')
fig.suptitle(f'Team 4-week report', fontsize=18, weight='bold')
facecolor = "white"
fig.set_facecolor(facecolor)

bottom = None

for i, param in enumerate(cols2inspect):
    color, fill, hatch, width = fill_design[i]

    x_values = relative_df.week.astype(float).values.copy()
    x_offset = 0.2

    if i == 3:
        x_values -= x_offset
    elif i == 5:
        x_values += x_offset

    ax.bar(
        x_values,
        relative_df[param],
        width=width,
        label=param,
        bottom=bottom,
        color=color,
        fill=fill,
        hatch=hatch,
    )
    if param == cols2inspect[0]:
        bottom = relative_df[param].values.copy()
    elif i < 3:
        bottom += relative_df[param].values

ax.set_ylim(ymin=0, ymax=6)
ax.yaxis.set_major_formatter(mtick.PercentFormatter(1.0))

x_tick_labels, y_tick_labels = ax.get_xticklabels(), ax.get_yticklabels()
for x_tick_label  in x_tick_labels:
    x_tick_label.set_fontsize(12)
for y_tick_label  in y_tick_labels:
    y_tick_label.set_fontsize(12)

ax.set_ylabel('Game Reference', fontsize=12, weight='bold')
ax.set_xlabel('Weeks', fontsize=12, weight='bold')

ax.legend(
    loc='upper center', labels=cols2inspect,
    bbox_to_anchor=(0.5, 1.05), ncol=3, fancybox=True, shadow=True,
    fontsize=10
)


ax.xaxis.set_major_locator(MaxNLocator(integer=True))
plt.xticks(rotation=12)
plt.show()

# 5. Player-Level Analysis and Session Reports

In this section, we'll focus on helping strength and conditioning coaches evaluate individual training sessions. To do this, we will:

- Extract the maximum values for each parameter for every player.
- Visualize the player’s performance in a single session.

---

In [ ]:
zoom_df.loc[:, 'acc_dec_cnt'] = zoom_df.acc_cnt + zoom_df.dec_cnt

In [ ]:
ind_week_4_df = zoom_df[(zoom_df.date >= date_from) & (zoom_df.date <= date_until)]

In [ ]:
def extract_max_for_player(all_sessions_df: pd.DataFrame, player_name: str, feature_name: str) -> float:
  player_session_df = all_sessions_df[all_sessions_df.player == player_name]
  return player_session_df[feature_name].max()

def extract_max_features(all_sessions_df: pd.DataFrame, players_list: list, param_name: str) -> list:
  max_features = []
  for p in players_list:
    value = extract_max_for_player(all_sessions_df, p, param_name)
    if value == 0:
      value = all_sessions_df[param_name].max()
    max_features.append(value)
  return max_features

In [ ]:
def get_colors_and_percentages(session_values: list, max_values: list) -> list:
  colors = []
  percentages = []

  COLOR_DICT = {
      'low': 'tomato',
      'neutral': 'steelblue',
      'high': 'forestgreen'
  }

  for val, max_v in zip(session_values, max_values):
    percentage = int(val / max_v * 100)
    LOW = 40
    HIGH = 80
    if percentage < LOW:
      color = COLOR_DICT['low']
    elif LOW < percentage < HIGH:
      color = 'steelblue'
    else:
      color = COLOR_DICT['high']
    colors.append(color)
    percentages.append(percentage)
  return colors, percentages

In [ ]:
TEXT_COLOR = 'lightgrey'
plt.rcParams['text.color'] = TEXT_COLOR
plt.rcParams['axes.labelcolor'] = TEXT_COLOR
plt.rcParams['xtick.color'] = TEXT_COLOR
plt.rcParams['ytick.color'] = TEXT_COLOR

In [ ]:
report_colums = ['total_distance_m', 'HI_m', 'acc_dec_cnt', 'player_load']

In [ ]:
session_date = ind_week_4_df.date.iloc[-1]
session_df = ind_week_4_df[ind_week_4_df.date == session_date]
players = session_df.player.unique()

num_columns = 4
num_players = players.shape[0]
fig, axs = plt.subplots(figsize=(10, 6), nrows=1, ncols=num_columns)
facecolor = "#00001a"
fig.set_facecolor(facecolor)


fig.suptitle(f'Session report {session_date}', fontsize=18, weight='bold')

ax_count = 0
for param, ax in zip(report_colums, axs.reshape(-1)):
  max_features = extract_max_features(zoom_df, players, param)

  separation_factor = 2

  session_param_values = session_df[param].values
  colors, percentages = get_colors_and_percentages(session_param_values, max_features)
  ax.barh(players, max_features, align='center', color='gray', height=0.6, label='Max')
  ax.barh(players, session_param_values, align='center', color=colors, height=0.4, label='Session')

  for cnt, info in enumerate(zip(session_param_values, percentages)):
    value, percentage = info
    ax.text(y=cnt - 0.1, x=value / 2, s=f'{percentage}%', fontsize=10)

  ax.yaxis.set_tick_params(labelsize=10)
  ax.xaxis.set_tick_params(labelsize=10)

  y_tick_labels = ax.get_yticklabels()
  for y_tick_label in y_tick_labels:
    y_tick_label.set_fontweight('bold')

  if ax_count % num_columns != 0:
    ax.axes.yaxis.set_ticklabels([])
  ax_count += 1
  ax.set_facecolor(facecolor)
  ax.set_title(param, fontsize=14)

Thank you for your attention and dedication throughout this process. Your efforts in analyzing and understanding the data will be crucial in improving performance and supporting the coaching staff. Keep up the great work!